## Main scraping code for GARLAND in cell below

I needed to do Garland's opinions separately because his sources were more disperate than the justices. I ultimately decided to only use his constitutional law opinions because those are obviously the most relevant to our needs for this project

In [44]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

In [58]:
from bs4 import BeautifulSoup
import requests
import re

def graboplinks():
    caselist = ["civil_rights","constitutional-law", "criminal-law", "administrative-law"]
    root_url = 'http://www.merrickgarlandproject.com/'
    all_links = {}
    
    for case_type in caselist:
        response = root_url + case_type
        
    #getting info from urls
        resp = requests.get(response)
        resp = BeautifulSoup(resp.text, "lxml")
        article = resp.find("article")
        
        
        #grabs only the links on the page with read opinion
        #blocking pdfs bc pdfs are lame
        for link in article.findAll("a", text=re.compile("Read Opinion")):
            url = link["href"]
            if not url.endswith("pdf"):
                response = requests.get(url)
                page = response.text
                soup = BeautifulSoup(page, "lxml")
                
                #goes to justia site if it's a justia link
                if re.search("justia", url):
                    pagesoup = soup.find_all(class_="tab-content")    
                    for element in pagesoup:
                        text = element.get_text()
                        dummytext = text + " "
                        if re.search("dissenting opinion filed by circuit judge garland", str(pagesoup).lower()):
                            opinion, dissent = text.split("GARLAND, Circuit Judge, dissenting")
                            all_links[url] = [case_type]
                            all_links[url].append("dissent")
                            all_links[url].append(dissent)
                        else:
                            all_links[url] = [case_type] 
                            all_links[url].append("opinion")
                            all_links[url].append(dummytext) 
                
                #goes to openjurist site if it's an openjurist link
                elif re.search("openjurist", url):          
                    pagesoup = soup.find_all(class_="node-content clear-block prose")    
                    for element in pagesoup:
                        text = element.get_text()
                        dummytext = text + " "
                        if re.search("dissenting opinion filed by circuit judge garland", str(pagesoup).lower()):
                            opinion, dissent = text.split("GARLAND, Circuit Judge, with whom")
                            all_links[url] = [case_type]
                            all_links[url].append("dissent")
                            all_links[url].append(dissent)
                        else:
                            all_links[url] = [case_type]
                            all_links[url].append("opinion")
                            all_links[url].append(dummytext)
                            
                elif re.search("leagle", url):          
                    pagesoup = soup.find_all(class_="decision-text-content")
                    dummytext = text + " "
                    for element in pagesoup:
                        text = element.get_text()
                        all_links[url] = [case_type]
                        all_links[url].append("opinion")
                        all_links[url].append(dummytext)
                #if link type not found, just append blank stuff
                
                elif re.search("titleii", url):          
                    article = soup.find("p")
                    article = article.get_text()
                    opinion, dissent = article.split("Garland, Circuit Judge, dissenting")
                    all_links[url] = [case_type]
                    all_links[url].append("dissent")
                    all_links[url].append(dissent)
                else:
                    all_links[url] = [case_type]
                    all_links[url].append("")
                    all_links[url].append("")        
                        

    return all_links

rawdata = graboplinks()

In [65]:
import pandas as pd

df = pd.DataFrame(rawdata)
df = df.transpose().reset_index()
#dfcasetext = df[df[""]]

def casetext(case):
    if re.search("casetext", case):
        return 1
    else:
        return 0
    
def cleanerup(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\\", " ")
    return text

df.columns = ["caseurl", "type", "ruled", "text"] 

df["iscasetext"] = df.caseurl.apply(casetext)
df["text"] = df.text.apply(cleanerup)
df.head(1)

,caseurl,type,ruled,text,iscasetext
0,http://law.justia.com/cases/federal/appellate-...,civil_rights,opinion,U.S. Court of Appeals for the District...,0


## Building a casetitle + year dataframe separately to merge

In [66]:
caselist = ["civil_rights","constitutional-law", "criminal-law", "administrative-law"]
root_url = 'http://www.merrickgarlandproject.com/'
all_links = []
titles = []
years = []
for case_type in caselist:
    response = root_url + case_type
    resp = requests.get(response)
    resp = BeautifulSoup(resp.text, "lxml")
    article = resp.find("article")
    for link in article.findAll("a", text=re.compile("Read Opinion")):
            url = link["href"]
            all_links.append(url)
    for title in resp.find_all("h3"):
        title = title.get_text().replace("\n", "")
        title, year = title.split("(" )
        year = year.replace(")", "")
        titles.append(title)
        years.append(year)
        
casetitles = pd.DataFrame(titles, all_links).reset_index()
casetitles["years"] = years
casetitles.columns = ["caseurl", "casetitle", "year"]
casetitles.head(1)  

,caseurl,casetitle,year
0,https://casetext.com/case/malik-v-dist-of-colu...,Malik v. District of Columbia,2009


In [68]:
dfwithtitles = pd.merge(casetitles, df, on="caseurl", how= "inner")

In [70]:
#splitting dfs into two to track down casetext docs in an easier format
df2, casetext = dfwithtitles[dfwithtitles["iscasetext"] == 0], dfwithtitles[dfwithtitles["iscasetext"] == 1]
casetext = casetext[casetext["type"] == "constitutional-law"]
#casetext
df2.to_pickle("Garland_firstpass.pickle")

## Lost a lot of constitutional law opinions because casetext isn't scrapeable. 
Manually grabbed a handful of links for constitutional law opinions from one of the cleaner sites

In [188]:
def grabconlaw():
    datum = {}
    links = ["http://www.leagle.com/decision/In%20FCO%2020140715151/MPOY%20v.%20RHEE",
            "http://www.leagle.com/decision/20081366532F3d834_11291/PARHAT%20v.%20GATES",
         "http://www.leagle.com/decision/20031385323F3d1062_11284/RANCHO%20VIEJO,%20LLC%20v.%20NORTON",
         "http://www.leagle.com/decision/In%20FDCO%2020160426869/IN%20RE%20ETHICON,%20INC.,%20PELVIC%20REPAIR%20SYSTEM%20PRODUCTS%20LIABILITY%20LITIGATION",
         "http://www.leagle.com/decision/In%20FCO%2020101018069/FREEDOM%20HOLDINGS,%20INC.%20v.%20CUOMO",
         "http://www.leagle.com/decision/19951435903FSupp532_11348/STATE%20OF%20NEW%20YORK%20BY%20VACCO%20v.%20REEBOK%20INTERN.%20LTD.",
         "http://www.leagle.com/decision/19921205959F2d246_11147/U.S.%20v.%20HARRIS"]
             
    for link in links:
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")    
        title = pageheader(soup)
        info = pagetext(soup)
        info.append(link)
        datum[title] = info
    return(datum)    
    
data = grabconlaw()    

In [181]:
def pageheader(soup):
    headers = soup.find_all("h1")
    for element in headers:
        header = element.get_text()
        header = header.replace("\n", "")
        header = header.replace("\t", "")
        title = header
    return title

In [187]:
def pagetext(soup):
    info = []
    
    pagesoup = soup.find_all(class_="decision-text-content")
    for element in pagesoup:
        text = element.get_text()
        dummytext = text + " "
        
        if re.search("dissenting opinion filed by circuit judge garland", str(pagesoup).lower()):
            opinion, dissent = text.split("GARLAND, Circuit Judge, dissenting")
            info.append("constitutional-law")
            info.append("dissent")
            info.append(dissent)
        else:
            info.append("constitutional-law")
            info.append("opinion")
            info.append(dummytext)
            
    return info

In [236]:
def casetext(case):
    if re.search("casetext", case):
        return 1
    else:
        return 0
    
def cleanerup(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\\", " ")
    text = text.lower()
    return text

## Making the two dataframes the same to concatenate them

In [196]:
df2 = pd.read_pickle("Garland_firstpass.pickle")

In [205]:
df2 = df2[df2["ruled"] != ""]
df2 = df2[['caseurl', 'casetitle', 'year', 'type', 'ruled', 'text']]

In [225]:
df["year"] = 0
df = df[['caseurl', 'casetitle', 'year', 'type', 'ruled', 'text']]

In [229]:
garland = pd.concat([df2, df])

In [232]:
garland.reset_index(inplace = True)

In [237]:
garland["casetitle"] = garland.casetitle.apply(cleanerup)
garland["text"] = garland.text.apply(cleanerup)

In [239]:
garland.to_pickle("fullgarland.pickle")

### Post process review
I found a corpus of data that educated lawers thought were relevant for Garland's qualifications to be a supreme court justice. On this site, these lawyers had compiled a list of urls from disperate sites with opinions and dissents of Garland. A huge challenge with this was to figure out how to break text up for topics I don't understand (I'm not a lawyer), but I was able to deduce what was going on in each document and figure out how to break up the texts based on patterns. 

__Next time__
-  I'd find one good website for all of this person's text! However, this was a really good learning process for BeautifulSoup because I'd had a hard time with that in the past.
-  I'd be sure to get the descriptive info at the same time as the body of text to prevent later headaches. 